In [2]:
from pymongo import MongoClient
import pandas as pd
import numpy as np

client = MongoClient('mongodb://localhost:27017/')
db = client['myDatabase']
posts = db.posts.find()

data = []
users = set()   
items = set()


for post in posts:
    post_id = str(post['_id'])
    for user_id in post['liked_by_users']:
        user_id_str = str(user_id)
        data.append({'user_id': user_id_str, 'post_id': post_id, 'rating': 1})
        users.add(user_id_str)
        items.add(post_id)


df = pd.DataFrame(data)


negative_data = []
n_negatives = 5  # Количество отрицательных образцов на каждый положительный

for index, row in df.iterrows():
    chosen_items = set(np.random.choice(list(items - {row['post_id']}), n_negatives, replace=False))
    for item in chosen_items:
        negative_data.append({'user_id': row['user_id'], 'post_id': item, 'rating': 0})

negative_df = pd.DataFrame(negative_data)
full_df = pd.concat([df, negative_df]).reset_index(drop=True)

print(full_df)


                      user_id                   post_id  rating
0    6630ddc193b8ddcb96a663e2  6630dfe693b8ddcb96a663e4       1
1    6630ddc193b8ddcb96a663e2  6630e08693b8ddcb96a663e6       1
2    6630ddc193b8ddcb96a663e2  6630e0c793b8ddcb96a663e8       1
3    6630e0fe93b8ddcb96a663e9  6630e13293b8ddcb96a663eb       1
4    662ff4f6bc9f94fc79d58881  6630e13293b8ddcb96a663eb       1
..                        ...                       ...     ...
331  6630ecb593b8ddcb96a6642b  6630ea3493b8ddcb96a66415       0
332  6630ecb593b8ddcb96a6642b  6630e5e593b8ddcb96a66409       0
333  6630ecb593b8ddcb96a6642b  6630e9bf93b8ddcb96a6640e       0
334  6630ecb593b8ddcb96a6642b  6630eb1193b8ddcb96a6641e       0
335  6630ecb593b8ddcb96a6642b  6630eaf793b8ddcb96a6641c       0

[336 rows x 3 columns]


In [3]:
df = full_df
df

,user_id,post_id,rating
0,6630ddc193b8ddcb96a663e2,6630dfe693b8ddcb96a663e4,1
1,6630ddc193b8ddcb96a663e2,6630e08693b8ddcb96a663e6,1
2,6630ddc193b8ddcb96a663e2,6630e0c793b8ddcb96a663e8,1
3,6630e0fe93b8ddcb96a663e9,6630e13293b8ddcb96a663eb,1
4,662ff4f6bc9f94fc79d58881,6630e13293b8ddcb96a663eb,1
...,...,...,...
331,6630ecb593b8ddcb96a6642b,6630ea3493b8ddcb96a66415,0
332,6630ecb593b8ddcb96a6642b,6630e5e593b8ddcb96a66409,0
333,6630ecb593b8ddcb96a6642b,6630e9bf93b8ddcb96a6640e,0
334,6630ecb593b8ddcb96a6642b,6630eb1193b8ddcb96a6641e,0


In [5]:
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import cross_validate
import pandas as pd

In [4]:
df

,user_id,post_id,rating
0,6630ddc193b8ddcb96a663e2,6630dfe693b8ddcb96a663e4,1
1,6630ddc193b8ddcb96a663e2,6630e08693b8ddcb96a663e6,1
2,6630ddc193b8ddcb96a663e2,6630e0c793b8ddcb96a663e8,1
3,6630e0fe93b8ddcb96a663e9,6630e13293b8ddcb96a663eb,1
4,662ff4f6bc9f94fc79d58881,6630e13293b8ddcb96a663eb,1
...,...,...,...
331,6630ecb593b8ddcb96a6642b,6630ea3493b8ddcb96a66415,0
332,6630ecb593b8ddcb96a6642b,6630e5e593b8ddcb96a66409,0
333,6630ecb593b8ddcb96a6642b,6630e9bf93b8ddcb96a6640e,0
334,6630ecb593b8ddcb96a6642b,6630eb1193b8ddcb96a6641e,0


In [13]:
from surprise import KNNWithMeans
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split

sim_options = {
    'name': 'pearson_baseline',
    'user_based': False,  # item-based подход
    'shrinkage': 100
}


model = KNNWithMeans(sim_options=sim_options)

reader = Reader(rating_scale=(0, 1))
data = Dataset.load_from_df(df[['user_id', 'post_id', 'rating']], reader)

trainset, testset = train_test_split(data, test_size=0.25)

model.fit(trainset)


predictions = model.test(testset)
for pred in predictions:
    print(pred)

from surprise import accuracy
accuracy.rmse(predictions)


Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
user: 6630ecb593b8ddcb96a6642b item: 6630e46293b8ddcb96a663fd r_ui = 0.00   est = 0.11   {'actual_k': 18, 'was_impossible': False}
user: 662ff4f6bc9f94fc79d58881 item: 6630ebf793b8ddcb96a6642a r_ui = 0.00   est = 0.03   {'actual_k': 31, 'was_impossible': False}
user: 6630ecb593b8ddcb96a6642b item: 6630e4b793b8ddcb96a66400 r_ui = 0.00   est = 0.04   {'actual_k': 16, 'was_impossible': False}
user: 6630eb4793b8ddcb96a66421 item: 6630e1e493b8ddcb96a663ed r_ui = 0.00   est = 0.05   {'actual_k': 3, 'was_impossible': False}
user: 6630e25d93b8ddcb96a663f0 item: 6630e2ba93b8ddcb96a663f2 r_ui = 1.00   est = 0.00   {'actual_k': 8, 'was_impossible': False}
user: 6630e47d93b8ddcb96a663fe item: 6630eba393b8ddcb96a66427 r_ui = 0.00   est = 0.10   {'actual_k': 4, 'was_impossible': False}
user: 6630e47d93b8ddcb96a663fe item: 6631491a93b8ddcb96a6642f r_ui = 0.00   est = 0.00   {'actual_k'

0.4639569923147151

In [29]:
from surprise import KNNWithMeans
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from surprise.accuracy import rmse

def detailed_predictions(predictions):
    print(f"{'User ID':>10} {'Item ID':>10} {'True Rating':>12} {'Pred Rating':>12} {'Error':>10}")
    print(predictions)
    for prediction in predictions:
        uid = prediction.uid
        iid = prediction.iid
        true_r = prediction.r_ui
        est = prediction.est
        error = true_r - est
        
        print(f"{uid:>10} {iid:>10} {true_r:>12} {est:>12.3f} {error:>10.3f}")

sim_options = {
    'name': 'pearson_baseline',
    'user_based': True,  # item-based подход
    'shrinkage': 1      # регуляризация для предотвращения деления на ноль
}

model = KNNWithMeans(sim_options=sim_options)

reader = Reader(rating_scale=(0, 1))
data = Dataset.load_from_df(df[['user_id', 'post_id', 'rating']], reader)

trainset, testset = train_test_split(data, test_size=0.25)

model.fit(trainset)

predictions = model.test(testset)

print("RMSE on test data:")
rmse(predictions)

print("\nDetailed predictions:")
detailed_predictions(predictions)


Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE on test data:
RMSE: 0.4562

Detailed predictions:
   User ID    Item ID  True Rating  Pred Rating      Error
[Prediction(uid='6630ecb593b8ddcb96a6642b', iid='6630e51f93b8ddcb96a66402', r_ui=0.0, est=0, details={'actual_k': 1, 'was_impossible': False}), Prediction(uid='662ff4f6bc9f94fc79d58881', iid='6630dfe693b8ddcb96a663e4', r_ui=0.0, est=0.0069444444444444475, details={'actual_k': 1, 'was_impossible': False}), Prediction(uid='6630e36f93b8ddcb96a663f7', iid='6630eb2e93b8ddcb96a66420', r_ui=0.0, est=0.0502483009657628, details={'actual_k': 3, 'was_impossible': False}), Prediction(uid='6630eb4793b8ddcb96a66421', iid='6630e13293b8ddcb96a663eb', r_ui=0.0, est=0.15833333333333333, details={'actual_k': 2, 'was_impossible': False}), Prediction(uid='6630e57793b8ddcb96a66405', iid='6630e3cd93b8ddcb96a663f9', r_ui=0.0, est=0.6895999245502403, details={'actual_k': 3, 'was_imp

In [31]:
def recommend_new_items(model, trainset, user_id, known_item_ids):
    """
    Функция для предсказания рейтингов неоцененных товаров для пользователя.

    :param model: Обученная модель предсказания.
    :param trainset: Обучающий набор данных, использованный для обучения модели.
    :param user_id: ID пользователя, для которого делаются предсказания.
    :param known_item_ids: Список ID товаров, которые пользователь уже оценил.
    :return: Словарь с ID товаров и предсказанными рейтингами, отсортированный по убыванию рейтинга.
    """
    # Получаем полный список товаров из trainset
    all_items = trainset.all_items()
    item_id_to_raw = trainset.to_raw_iid
    
    # Фильтруем и преобразуем индексы в исходные ID товаров
    unknown_items = [item_id_to_raw(i) for i in all_items if item_id_to_raw(i) not in known_item_ids]
    
    predictions = {}
    for item_id in unknown_items:
        pred = model.predict(user_id, item_id)
        predictions[item_id] = pred.est
    
    # Сортируем предсказания по убыванию предсказанного рейтинга
    sorted_predictions = sorted(predictions.items(), key=lambda x: x[1], reverse=True)
    
    return sorted_predictions

# Пример использования
user_id = '6630ddc193b8ddcb96a663e2'
known_item_ids = ['6630e4b793b8ddcb96a66400', '6630e51f93b8ddcb96a66402', '6630e55e93b8ddcb96a66404']
recommendations = recommend_new_items(model, trainset, user_id, known_item_ids)

print("Рекомендации для пользователя:", recommendations)


Рекомендации для пользователя: [('6630dfe693b8ddcb96a663e4', 1), ('6630e35593b8ddcb96a663f6', 1), ('6630e9bf93b8ddcb96a6640e', 1), ('6630e0c793b8ddcb96a663e8', 1), ('6630e9dc93b8ddcb96a66410', 1), ('6630e08693b8ddcb96a663e6', 0.8612766897804034), ('6630e9fc93b8ddcb96a66412', 0.5125), ('6630e2ea93b8ddcb96a663f4', 0.5), ('6630e2ba93b8ddcb96a663f2', 0.2), ('6630e1e493b8ddcb96a663ed', 0.2), ('6630e5e593b8ddcb96a66409', 0.2), ('6630e13293b8ddcb96a663eb', 0.2), ('6631491a93b8ddcb96a6642f', 0.2), ('6630eccf93b8ddcb96a6642d', 0.2), ('6630e24193b8ddcb96a663ef', 0.2), ('6630e5af93b8ddcb96a66407', 0.2), ('6630eb6f93b8ddcb96a66423', 0.022536271835580768), ('6630eb8993b8ddcb96a66425', 0.022536271835580768), ('6630eba393b8ddcb96a66427', 0.0204880796160839), ('6630eab793b8ddcb96a66419', 0.012500000000000011), ('6630eb1193b8ddcb96a6641e', 0.0020496042235790823), ('6630e42593b8ddcb96a663fb', 0.0017559912686024859), ('6630eb2e93b8ddcb96a66420', 0.0011163229185381096), ('6630e46293b8ddcb96a663fd', 0.0011

In [38]:
import os
from surprise import dump

model_path = 'models/recommendation_model.pkl'
dump.dump(model_path, algo=model)